In [19]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

In [20]:
# Configs
DATA_PATH = "Database/img/"
SEQ_LEN = 300   # 3s @ 10ms = 300 steps
INPUT_DIM = 6
BATCH_SIZE = 32
EPOCHS = 20
LR = 1e-3

In [21]:
# Load and preprocess data
sequences = []
labels = []

for filename in os.listdir(DATA_PATH):
    if filename.endswith('.csv'):
        path = os.path.join(DATA_PATH, filename)
        df = pd.read_csv(path, header=None, dtype=str)

        df = df.astype(str).apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))
        df = df.replace(r'[^\d\.-]', '', regex=True)
        df = df.apply(pd.to_numeric, errors='coerce')
        df = df.dropna()

        if df.shape[0] >= SEQ_LEN:
            df = df.iloc[:SEQ_LEN, :]
        else:
            pad = pd.DataFrame(np.zeros((SEQ_LEN - df.shape[0], df.shape[1])))
            df = pd.concat([df, pad], ignore_index=True)

        sequences.append(df.values.astype(np.float32))
        labels.append(filename.split('_')[0])

In [22]:
# Encode labels
label_to_idx = {label: idx for idx, label in enumerate(sorted(set(labels)))}
y_indices = [label_to_idx[label] for label in labels]

In [23]:
# Convert to tensors
X = torch.tensor(np.stack(sequences))  # [N, 300, 6]
y = torch.tensor(y_indices, dtype=torch.long)

In [24]:
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: torch.Size([105, 300, 6])
y shape: torch.Size([105])


In [25]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [26]:
# Dataset and DataLoader
class GestureDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_loader = DataLoader(GestureDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(GestureDataset(X_test, y_test), batch_size=BATCH_SIZE)




In [27]:
# LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        return self.fc(h_n.squeeze(0))

model = LSTMClassifier(INPUT_DIM, 128, len(label_to_idx))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

LSTMClassifier(
  (lstm): LSTM(6, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=5, bias=True)
)

In [28]:
# Training
optimizer = optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {total_loss:.4f}")

Epoch 1/20 - Loss: 4.9343
Epoch 2/20 - Loss: 4.5961
Epoch 3/20 - Loss: 4.3409
Epoch 4/20 - Loss: 4.1330
Epoch 5/20 - Loss: 3.9388
Epoch 6/20 - Loss: 3.8300
Epoch 7/20 - Loss: 3.6616
Epoch 8/20 - Loss: 3.5322
Epoch 9/20 - Loss: 3.4333
Epoch 10/20 - Loss: 3.3316
Epoch 11/20 - Loss: 3.1975
Epoch 12/20 - Loss: 3.1377
Epoch 13/20 - Loss: 3.0271
Epoch 14/20 - Loss: 2.9437
Epoch 15/20 - Loss: 2.8199
Epoch 16/20 - Loss: 2.7415
Epoch 17/20 - Loss: 2.6020
Epoch 18/20 - Loss: 2.5401
Epoch 19/20 - Loss: 2.4676
Epoch 20/20 - Loss: 2.3707


In [30]:
# Evaluation
model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        outputs = model(batch_X)
        preds = torch.argmax(outputs, dim=1).cpu()
        all_preds.extend(preds.numpy())
        all_targets.extend(batch_y.numpy())

acc = accuracy_score(all_targets, all_preds)
print(f"\nTest Accuracy: {acc:.4f}")

# Generate target names from label_to_idx
idx_to_label = {v: k for k, v in label_to_idx.items()}
target_names = [idx_to_label[i] for i in range(len(idx_to_label))]

print("\nClassification Report:\n", classification_report(all_targets, all_preds, target_names=target_names))





Test Accuracy: 0.6667

Classification Report:
               precision    recall  f1-score   support

           A       0.50      0.50      0.50         4
           B       1.00      0.50      0.67         4
           C       0.33      0.50      0.40         4
           D       1.00      0.80      0.89         5
           E       0.80      1.00      0.89         4

    accuracy                           0.67        21
   macro avg       0.73      0.66      0.67        21
weighted avg       0.74      0.67      0.68        21



In [34]:
# Predict and print results for the test set
print("\nPredictions on Test Set:")
for i in range(len(all_preds)):
    true_label = idx_to_label[all_targets[i]]
    predicted_label = idx_to_label[all_preds[i]]
    print(f"{i+1}: True = {true_label}, Predicted = {predicted_label}")


Predictions on Test Set:
1: True = E, Predicted = E
2: True = A, Predicted = C
3: True = C, Predicted = C
4: True = A, Predicted = A
5: True = D, Predicted = D
6: True = D, Predicted = D
7: True = C, Predicted = C
8: True = D, Predicted = D
9: True = D, Predicted = D
10: True = E, Predicted = E
11: True = A, Predicted = A
12: True = B, Predicted = B
13: True = E, Predicted = E
14: True = C, Predicted = A
15: True = B, Predicted = C
16: True = B, Predicted = B
17: True = B, Predicted = A
18: True = A, Predicted = C
19: True = C, Predicted = E
20: True = D, Predicted = C
21: True = E, Predicted = E
